In [1]:
import sys
import numpy as np
import pandas as pd

from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import accuracy_score,roc_curve,auc,precision_score,recall_score

from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from expectation_reflection import ExpectationReflection

import expectation_reflection_cv as ERCV
#import expectation_reflection_cvmc as ERCVMC
from function import split_train_test,make_data_balance

In [2]:
np.random.seed(1)

In [3]:
def inference(X_train,y_train,X_test,y_test):     
    n = X_train.shape[1]
    m = len(np.unique(y_train))

    l2 = np.logspace(-5,1,20,base=10.0)
    #l2 = [0.,0.00001,0.0001,0.001,0.01,0.1,1.]
    nl2 = len(l2)
              
    kf = 3   
    kfold = KFold(n_splits=kf,shuffle=False,random_state=1)

    # predict with ER
    #if m == 2:
    h01 = np.zeros(kf)
    w1 = np.zeros((kf,n))
    acc1 = np.zeros(kf)
    
    h0 = np.zeros(nl2)
    w = np.zeros((nl2,n))
    acc = np.zeros(nl2)            
    for il2 in range(nl2):            
        for i,(train_index,val_index) in enumerate(kfold.split(y_train)):
            X_train1, X_val = X_train[train_index], X_train[val_index]
            y_train1, y_val = y_train[train_index], y_train[val_index]
            
            #h01[i],w1[i,:] = ERCV.fit(X_train1,y_train1,X_val,y_val,niter_max=1000,l2=l2[il2])
            model = LogisticRegression(penalty='l2',C=1./l2[il2],solver='liblinear',multi_class='auto')
            model.fit(X_train1,y_train1)
            
            h01[i] = model.intercept_
            w1[i,:] = model.coef_
            y_val_pred,p_val_pred = ERCV.predict(X_val,h01[i],w1[i])
            
            #acc1[i] = accuracy_score(y_val,y_val_pred)
            acc1[i] = ((p_val_pred - y_val)**2).mean()
                                        
        h0[il2] = h01.mean(axis=0)
        w[il2,:] = w1.mean(axis=0)
        acc[il2] = acc1.mean()
    
    il2_select = np.argmin(acc)
    #print('l2:',l2[il2_select])
    y_pred,p_pred = ERCV.predict(X_test,h0[il2_select],w[il2_select,:])

    # entire training set:
    #model = ExpectationReflection(niter_max=1000,l2=l2[il2_select])
    #model.fit(X_train,y_train)
    #y_pred = model.predict(X_test)
                                        
    accuracy = accuracy_score(y_test,y_pred)
    cost = ((p_pred - y_test)**2).mean()
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
  
    # Compute ROC curve and ROC area for each class
    fpr, tpr,thresholds = roc_curve(y_test, p_pred, drop_intermediate=False)
    roc_auc= auc(fpr, tpr)

    #print(len(p_pred),len(y_test),len(fpr),len(tpr))                       

    return accuracy,roc_auc,precision,recall,cost

In [4]:
def compare_inference(X,y,train_size):
    npred = 5
    acc = np.zeros(npred)
    roc_auc = np.zeros(npred)
    precision = np.zeros(npred)
    recall = np.zeros(npred)
    cost = np.zeros(npred)
    for ipred in range(npred):
        X_train,X_test,y_train,y_test = split_train_test(X,y,train_size,test_size=0.7)

        #X_train = MinMaxScaler().fit_transform(X_train)
        #X_test = MinMaxScaler().fit_transform(X_test)
        
        # 2019.07.15
        #acc[ipred],cost[ipred],roc_auc[ipred] = inference(X_train,y_train,X_test,y_test)
        acc[ipred],roc_auc[ipred],precision[ipred],recall[ipred],cost[ipred] \
        = inference(X_train,y_train,X_test,y_test)
            
    #return acc.mean(axis=0),acc.std(),cost.mean(),cost.std(),roc_auc.mean(),roc_auc.std()
    return acc.mean(axis=0),roc_auc.mean(),precision.mean(),recall.mean(),cost.mean(),\
           acc.std(),roc_auc.std(),precision.std(),recall.std(),cost.std()

In [5]:
data_name_list = ['nki','peptide','amputation','wisconsin','heat','hc','paradox','school','stigma','ef',\
                 'smok','coag','anemia','cervix','mental','dement','septic','coimbra','cryo','somerville',\
                  'hcc','heart','liver','language']
n_data = len(data_name_list)

for data_id in range(n_data):
    data_name = data_name_list[data_id]
    # load data
    X = np.loadtxt('data_sets/%s_x.txt'%data_name)
    y = np.loadtxt('data_sets/%s_y.txt'%data_name)
    #print('ini:',np.unique(y,return_counts=True))

    X,y = make_data_balance(X,y)
    #print('balance:',np.unique(y,return_counts=True))

    X, y = shuffle(X, y)
    X = MinMaxScaler().fit_transform(X)                  

    list_train_size = [0.3]
    n_size = len(list_train_size)
    acc = np.zeros((n_size,10)) # 0: acc, 1: acc_std, 2: cost, 3: cost_std
    for i,train_size in enumerate(list_train_size):
        acc[i,:] = compare_inference(X,y,train_size)
        print(i,train_size,acc[i,:])

    np.savetxt('result/%s_acc_LR.dat'%data_name,acc,fmt='%f')

0 0.3 [0.71509434 0.798932   0.70714413 0.74716981 0.19212506 0.03499479
 0.03263744 0.0504792  0.04861924 0.0142433 ]
0 0.3 [0.94375    0.99609375 0.94489164 0.95       0.04133863 0.04592793
 0.00494106 0.05773475 0.1        0.02533395]
0 0.3 [0.68       0.73955556 0.6695911  0.74666667 0.23797996 0.04
 0.01529302 0.0551695  0.07774603 0.01388014]
0 0.3 [0.95       0.98840394 0.95945847 0.94054054 0.03982166 0.01234842
 0.00379997 0.02576359 0.00787966 0.00747211]
0 0.3 [0.56949153 0.63125539 0.56894734 0.6440678  0.24529255 0.03989357
 0.03379668 0.05133055 0.10984306 0.00700306]
0 0.3 [0.78857143 0.84081633 0.77917657 0.81714286 0.17187854 0.0427618
 0.04703344 0.05595942 0.10932072 0.03173936]
0 0.3 [0.89318182 0.91683884 0.97948718 0.80454545 0.09349406 0.00556702
 0.02161042 0.02512297 0.03082877 0.00784088]
0 0.3 [0.87021277 0.92901766 0.85109554 0.89787234 0.10813694 0.00796097
 0.01574801 0.01296295 0.01592195 0.00643077]
0 0.3 [0.60539887 0.64334856 0.60623043 0.60225624 0.23